In [2]:
data_path = "Conway_GPT_v8\\"

test_file = data_path + "conway_test_states_32by32_20240716_151502.csv"

import pandas as pd
df_test = pd.read_csv(test_file)

future_steps_list = [2]
start_char = '@'
end_char = '$'
mask_char = ['_']
# Function to generate data
def generate_data(df, future_steps):
    X_data = []
    for i in range(len(df['State 1'])):
        future_state_col = f'State {future_steps}'
        if future_state_col in df.columns:
            str_ = f"{start_char}PredictNextState<{df['State 1'][i]}> [{df[future_state_col][i]}]{end_char}"
            X_data.append(str_)
    return X_data
X_data_test = {steps: generate_data(df_test, steps) for steps in future_steps_list}


In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [7]:
X_data_test_new=X_data_test[2]

In [22]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from x_transformers import TransformerWrapper, Decoder
from x_transformers.autoregressive_wrapper import AutoregressiveWrapper
import imageio
from datetime import datetime

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Define necessary classes and functions (Tokenizer, empty_cuda_cache, load_model, extract_task, extract_sample)
# ... (unchanged code from your previous example)

def autoregressive_generation(model, tokenizer, initial_input_str, num_steps, temp):
    current_input = extract_task(initial_input_str, end_task_token='>')
    generated_states = []

    for _ in range(num_steps):
        current_input_tensor = torch.Tensor(tokenizer.texts_to_sequences(current_input, do_padding=False)).to(device)
        current_input_tensor = current_input_tensor.transpose(0, 1).long()

        with torch.no_grad():
            sample = model.generate(
                prompts=current_input_tensor,
                seq_len=generate_length,
                temperature=temp,
                cache_kv=True
            )

        output_str = extract_sample(tokenizer.sequences_to_texts(sample[:1])[0])
        print(f"Output: {output_str}")
        generated_states.append(output_str)

        # Update the input for the next step
        current_input = f"@PredictNextState<{output_str}>"

        del sample, current_input_tensor
        torch.cuda.empty_cache()

    return generated_states

def display_and_save_images(initial_input, data_list, temp, base_dir="output_gifs"):
    num_states = len(data_list) + 1  # Include the initial input as the first state
    images = []

    plt.figure(figsize=(15, 5))

    # Convert and plot the initial input state
    initial_image = convert_to_square_image(initial_input)
    plt.subplot(1, num_states, 1)
    plt.title(f"State 1")
    plt.imshow(1 - initial_image, cmap='gray', vmin=0, vmax=1)  # Corrected color inversion
    plt.axis('off')
    images.append(1 - initial_image)

    # Convert and plot the subsequent states
    for i, data in enumerate(data_list):
        image = convert_to_square_image(data)
        plt.subplot(1, num_states, i + 2)
        plt.title(f"State {i + 2}")
        plt.imshow(1 - image, cmap='gray', vmin=0, vmax=1)  # Corrected color inversion
        plt.axis('off')
        images.append(1 - image)

    plt.show()

    temp_folder = os.path.join(base_dir, f"temp_{temp}")
    os.makedirs(temp_folder, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    gif_path = os.path.join(temp_folder, f"sequence_{timestamp}.gif")
    mp4_path = os.path.join(temp_folder, f"sequence_{timestamp}.mp4")

    # Ensure all images are in RGB format for MP4
    images_rgb = [np.stack([img, img, img], axis=-1) * 255 for img in images]  # Scale to 255 for OpenCV

    # Save as GIF
    imageio.mimsave(gif_path, images_rgb, format='GIF', duration=200, loop=0)

    # Save as MP4 using OpenCV
    height, width = images_rgb[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(mp4_path, fourcc, 5, (width, height))  # 5 fps

    for img in images_rgb:
        out.write(np.uint8(img))  # Convert to uint8 for OpenCV

    out.release()

    print(f"Saved ground truth GIF at {gif_path}")
    print(f"Saved ground truth MP4 at {mp4_path}")

def convert_to_square_image(state):
    length = len(state)
    size = int(np.sqrt(length))
    assert size * size == length, "State length is not a perfect square"
    image = np.array([int(pixel) for pixel in state]).reshape((size, size))
    return image

def parse_and_convert_ground_truth(df, index):
    data = df.iloc[index]['states']
    states = data.split(',')
    return states

def display_and_save_ground_truth_gif(df, row_index, base_dir="ground_truth_gifs", scale_factor=10):
    states = df.iloc[row_index].tolist()
    images = [convert_to_square_image(state) for state in states]

    # Increase the resolution of the images for GIF and video
    images_resized = [cv2.resize(img, (img.shape[1] * scale_factor, img.shape[0] * scale_factor), interpolation=cv2.INTER_NEAREST) for img in images]

    # Invert colors: making the cells black (value 0) and the background white (value 255)
    images_inverted = [255 - (img * 255).astype(np.uint8) for img in images_resized]  # Convert binary 0/1 to 0/255, invert, and cast to uint8

    # Convert to RGB format
    images_rgb = [np.stack([img, img, img], axis=-1) for img in images_inverted]  # RGB stack
    
    # Save as GIF
    gif_path = os.path.join(base_dir, f"ground_truth_row_{row_index}.gif")
    imageio.mimsave(gif_path, images_rgb, format='GIF', duration=200, loop=0)
    
    # Save as MP4 using OpenCV
    mp4_path = os.path.join(base_dir, f"ground_truth_row_{row_index}.mp4")
    height, width = images_rgb[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(mp4_path, fourcc, 5, (width, height))  # 5 fps

    for img in images_rgb:
        out.write(img)

    out.release()

    print(f"Saved ground truth GIF at {gif_path}")
    print(f"Saved ground truth MP4 at {mp4_path}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [23]:
for index in range(len(X_data_test_new)):
    print(index)
    display_and_save_ground_truth_gif(df_test,index)
    

0
Saved ground truth GIF at ground_truth_gifs\ground_truth_row_0.gif
Saved ground truth MP4 at ground_truth_gifs\ground_truth_row_0.mp4
1
Saved ground truth GIF at ground_truth_gifs\ground_truth_row_1.gif
Saved ground truth MP4 at ground_truth_gifs\ground_truth_row_1.mp4
2
Saved ground truth GIF at ground_truth_gifs\ground_truth_row_2.gif
Saved ground truth MP4 at ground_truth_gifs\ground_truth_row_2.mp4
3
Saved ground truth GIF at ground_truth_gifs\ground_truth_row_3.gif
Saved ground truth MP4 at ground_truth_gifs\ground_truth_row_3.mp4
4
Saved ground truth GIF at ground_truth_gifs\ground_truth_row_4.gif
Saved ground truth MP4 at ground_truth_gifs\ground_truth_row_4.mp4
5
Saved ground truth GIF at ground_truth_gifs\ground_truth_row_5.gif
Saved ground truth MP4 at ground_truth_gifs\ground_truth_row_5.mp4
6
Saved ground truth GIF at ground_truth_gifs\ground_truth_row_6.gif
Saved ground truth MP4 at ground_truth_gifs\ground_truth_row_6.mp4
7
Saved ground truth GIF at ground_truth_gifs\gr

In [24]:
print(df_test.columns)

Index(['State 1', 'State 2', 'State 3', 'State 4', 'State 5', 'State 6',
       'State 7', 'State 8', 'State 9', 'State 10'],
      dtype='object')


## Autoregressive generation for Epoch 16

In [36]:
import os
import numpy as np
import pandas as pd
import torch
from datetime import datetime

def save_data_to_csv(df, row_indices, autoreg_list, temp, base_dir="Autoregressive_autoregressor"):
    """
    Saves the ground truth and autoregressively generated data into a CSV file.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the ground truth data.
    row_indices (list of int): List of indices corresponding to the rows in the DataFrame to extract ground truth states.
    autoreg_list (list of list of str): List containing lists of string data representing the autoregressively generated states.
    temp (float): The temperature used for generating the states.
    base_dir (str): Base directory to save the CSV file.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(base_dir, exist_ok=True)
    
    # Prepare data for saving
    data = []
    for i, row_index in enumerate(row_indices):
        gt_states = parse_and_convert_ground_truth(df, row_index)
        autoreg_states = autoreg_list[i]
        
        # Ensure the initial state is included in both rows
        initial_state = gt_states[0]
        
        gt_row = [initial_state] + gt_states[1:]  # Ground truth row
        autoreg_row = [initial_state] + autoreg_states  # Autoregressive row

        data.append(["Ground Truth"] + gt_row)
        data.append(["Autoregressive"] + autoreg_row)
    
    # Convert to DataFrame for easier saving
    columns = ["Type"] + [f"State {i + 1}" for i in range(len(gt_row))]
    data_df = pd.DataFrame(data, columns=columns)
    
    # Generate a unique CSV filename
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    csv_path = os.path.join(base_dir, f"data_temp_{temp}_{timestamp}.csv")
    
    # Save the DataFrame to a CSV file
    data_df.to_csv(csv_path, index=False)
    print(f"Saved data to CSV at {csv_path}")

def autoregressive_generation(model, tokenizer, initial_input_str, num_steps, temp):
    current_input = extract_task(initial_input_str, end_task_token='>')
    generated_states = []

    for _ in range(num_steps):
        current_input_tensor = torch.Tensor(tokenizer.texts_to_sequences(current_input, do_padding=False)).to(device)
        current_input_tensor = current_input_tensor.transpose(0, 1).long()

        with torch.no_grad():
            sample = model.generate(
                prompts=current_input_tensor,
                seq_len=generate_length,
                temperature=temp,
                cache_kv=True
            )

        output_str = extract_sample(tokenizer.sequences_to_texts(sample[:1])[0])
        print(f"Output: {output_str}")
        generated_states.append(output_str)

        # Update the input for the next step
        current_input = f"@PredictNextState<{output_str}>"

        del sample, current_input_tensor
        torch.cuda.empty_cache()

    return generated_states


# Assuming X_data_test[2] is already defined
X_data_test_new = X_data_test[2]

# Calculate max_length and generate_length dynamically
max_length = len(X_data_test_new[0])
generate_length = max_length - len("@PredictNextState<>") - (32 * 32)

# Initialize tokenizer
tokenizer_X = Tokenizer(max_length, device)

# Load the model for epoch 50
epoch = 16
model_path = f"model_parameters\\07_22_2024_Conway_2_State_Jump_Rot_Pos_On_Masking_On_Broad_Entrpoy_Homog_2024-07-23 10-37-31\\Conway_GPT_v7_epoch_{epoch}.pt"
model = load_model(model_path, max_length, num_words=256)

# Define the temperatures
temperatures = [0, 0.25, 0.5, 0.75, 1]

# Generate and save the states
num_steps = 9  # Number of steps to generate autoregressively
ground_truth_df = df_test  # Use the df_test DataFrame for ground truth data

for temp in temperatures:
    print(f"Generating for temperature {temp}...")
    
    autoregressive_states = []
    row_indices = []
    
    for i, item in enumerate(X_data_test_new):
        # Take the initial input from the test data
        initial_input_str = str(item)
        
        # Generate subsequent states
        generated_states = autoregressive_generation(model, tokenizer_X, initial_input_str, num_steps, temp)
        autoregressive_states.append(generated_states)
        row_indices.append(i)  # Assuming the index in X_data_test corresponds to the DataFrame row
        
        del generated_states
        torch.cuda.empty_cache()
    
    # Save the data for the current temperature
    save_data_to_csv(ground_truth_df, row_indices, autoregressive_states, temp)

    del autoregressive_states
    torch.cuda.empty_cache()


Model loaded from C:\Users\jaime\ML_Playground_1\model_parameters\07_22_2024_Conway_2_State_Jump_Rot_Pos_On_Masking_On_Broad_Entrpoy_Homog_2024-07-23 10-37-31\Conway_GPT_v7_epoch_16.pt
Generating for temperature 0...
Output: 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

## Gif Generation for Epoch 16

In [18]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import seaborn as sns
from PIL import Image, ImageDraw, ImageFont

def create_gifs_from_csv(csv_path_pattern, temp, output_dir="Figures", fps=5):
    """
    Creates looping GIFs from the CSV file, showing the evolution of Ground Truth (GT), Predicted (Pred),
    and Discrepancy states together.

    Parameters:
    csv_path_pattern (str): Pattern to match the CSV files.
    temp (float): The temperature associated with the data.
    output_dir (str): Directory to save the generated GIFs.
    fps (int): Frames per second for the GIF.
    """
    # Use glob to find the file that matches the pattern
    csv_files = glob.glob(csv_path_pattern)
    
    if not csv_files:
        print(f"No CSV files found for temperature {temp}")
        return
    
    # Assuming only one file matches the pattern
    csv_path = csv_files[0]
    
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Determine the number of states and samples
    num_columns = len([col for col in df.columns if "State" in col])
    num_samples = df.shape[0] // 2  # Since we have GT and Pred rows, divide by 2
    
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_samples):
        frames = []
        for j in range(num_columns):
            fig, axes = plt.subplots(3, 1, figsize=(3, 9), dpi=300)

            gt_row = df.iloc[2 * i, 1:]  # Skip the first column which is 'Type'
            pred_row = df.iloc[2 * i + 1, 1:]

            # GT image
            plot_image(axes[0], gt_row.iloc[j], f"GT", "gray")
            # Predicted image
            plot_image(axes[1], pred_row.iloc[j], f"Predicted", "gray")
            # Discrepancy image
            discrepancy_matrix = calculate_discrepancy_matrix(gt_row.iloc[j], pred_row.iloc[j])
            plot_discrepancy(axes[2], discrepancy_matrix, f"Discrepancy")

            # Add titles
            fig.suptitle(f"Sample {i + 1} | Temperature: {temp} | Epoch: 16/50", fontsize=10)

            plt.tight_layout(pad=2.0)
            
            # Save the figure to a temporary file
            temp_file = os.path.join(output_dir, f"temp_frame_{j}.png")
            plt.savefig(temp_file, dpi=300)
            plt.close(fig)
            
            # Add the image to the frame list
            frames.append(Image.open(temp_file))

        # Create the GIF
        gif_path = os.path.join(output_dir, f"evolution_temp_{temp}_sample_{i + 1}.gif")
        frames[0].save(gif_path, save_all=True, append_images=frames[1:], duration=1000 // fps, loop=0)
        print(f"GIF saved at {gif_path}")

        # Clean up temporary files
        for frame in frames:
            os.remove(frame.filename)

def plot_image(ax, state, ylabel, cmap):
    """
    Plots an image from the state data.

    Parameters:
    ax (Axes): The axes to plot on.
    state (str): The state as a string of 0s and 1s.
    ylabel (str): The label for the y-axis.
    cmap (str): The color map for the image.
    """
    ax.set_ylabel(ylabel, fontsize=10, rotation=90, labelpad=10, va='center')
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Turn off ticks and labels
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)  # Set the width of the spines
    if not is_perfect_square(len(state)) or not check_state_length(state):
        ax.text(0.5, 0.5, "DATA NOT SQUARE", fontsize=10, ha='center', va='center', rotation=45)
    else:
        ax.imshow(1 - convert_to_square_image(state, cell_size=10), cmap=cmap, vmin=0, vmax=1)

def plot_discrepancy(ax, discrepancy_matrix, ylabel):
    """
    Plots the discrepancy matrix.

    Parameters:
    ax (Axes): The axes to plot on.
    discrepancy_matrix (ndarray): The discrepancy matrix to plot.
    ylabel (str): The label for the y-axis.
    """
    sns.heatmap(discrepancy_matrix, cmap="plasma", cbar=False, square=True, ax=ax)
    ax.set_ylabel(ylabel, fontsize=10, rotation=90, labelpad=10, va='center')
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Turn off ticks and labels

def calculate_discrepancy_matrix(gt_state, pred_state):
    """
    Calculate the discrepancy matrix between GT and Predicted states.

    Parameters:
    gt_state (str): The ground truth state.
    pred_state (str): The predicted state.

    Returns:
    np.ndarray: The discrepancy matrix.
    """
    if len(gt_state) == len(pred_state):
        discrepancy_matrix = np.array([
            1 if gt_state[k] != pred_state[k] else 0
            for k in range(len(gt_state))
        ]).reshape((32, 32))
    else:
        print('not square!')
        discrepancy_matrix = np.full((32, 32), 0.5)  # Handle error gracefully
        print(discrepancy_matrix)
    
    return discrepancy_matrix

def convert_to_square_image(state, cell_size=1):
    """
    Converts a string state representation into a square image with adjustable cell size.
    
    Parameters:
    state (str): The state as a string of 0s and 1s.
    cell_size (int): The size of each cell in pixels. Default is 1.
    
    Returns:
    2D numpy array representing the square image with adjusted cell size.
    """
    length = len(state)
    size = int(np.sqrt(length))
    assert size * size == length, "State length is not a perfect square"
    image = np.array([int(pixel) for pixel in state]).reshape((size, size))
    
    # Adjust the cell size by repeating the rows and columns
    image = np.kron(image, np.ones((cell_size, cell_size)))
    
    return image

def is_perfect_square(length):
    """
    Check if a number is a perfect square.
    """
    size = int(np.sqrt(length))
    return size * size == length

def check_state_length(state):
    """
    Check if the state has 1024 entries.
    """
    return len(state) == 1024

# Example usage
temperatures = [0, 0.25, 0.5, 0.75, 1]
base_dir = "Autoregressive_autoregressor"

for temp in temperatures:
    csv_path_pattern = os.path.join(base_dir, f"data_temp_{temp}_*.csv")
    create_gifs_from_csv(csv_path_pattern, temp)


GIF saved at Figures\evolution_temp_0_sample_1.gif
GIF saved at Figures\evolution_temp_0_sample_2.gif
GIF saved at Figures\evolution_temp_0_sample_3.gif
GIF saved at Figures\evolution_temp_0_sample_4.gif
GIF saved at Figures\evolution_temp_0_sample_5.gif
GIF saved at Figures\evolution_temp_0_sample_6.gif
GIF saved at Figures\evolution_temp_0_sample_7.gif
GIF saved at Figures\evolution_temp_0_sample_8.gif
GIF saved at Figures\evolution_temp_0_sample_9.gif
GIF saved at Figures\evolution_temp_0_sample_10.gif
GIF saved at Figures\evolution_temp_0.25_sample_1.gif
GIF saved at Figures\evolution_temp_0.25_sample_2.gif
GIF saved at Figures\evolution_temp_0.25_sample_3.gif
GIF saved at Figures\evolution_temp_0.25_sample_4.gif
GIF saved at Figures\evolution_temp_0.25_sample_5.gif
GIF saved at Figures\evolution_temp_0.25_sample_6.gif
GIF saved at Figures\evolution_temp_0.25_sample_7.gif
GIF saved at Figures\evolution_temp_0.25_sample_8.gif
GIF saved at Figures\evolution_temp_0.25_sample_9.gif
GIF

## Autoregressive generation for Epoch 50

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from datetime import datetime

def save_data_to_csv(df, row_indices, autoreg_list, temp, base_dir="Autoregressive_autoregressor"):
    """
    Saves the ground truth and autoregressively generated data into a CSV file.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the ground truth data.
    row_indices (list of int): List of indices corresponding to the rows in the DataFrame to extract ground truth states.
    autoreg_list (list of list of str): List containing lists of string data representing the autoregressively generated states.
    temp (float): The temperature used for generating the states.
    base_dir (str): Base directory to save the CSV file.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(base_dir, exist_ok=True)
    
    # Prepare data for saving
    data = []
    for i, row_index in enumerate(row_indices):
        gt_states = parse_and_convert_ground_truth(df, row_index)
        autoreg_states = autoreg_list[i]
        
        # Ensure the initial state is included in both rows
        initial_state = gt_states[0]
        
        gt_row = [initial_state] + gt_states[1:]  # Ground truth row
        autoreg_row = [initial_state] + autoreg_states  # Autoregressive row

        data.append(["Ground Truth"] + gt_row)
        data.append(["Autoregressive"] + autoreg_row)
    
    # Convert to DataFrame for easier saving
    columns = ["Type"] + [f"State {i + 1}" for i in range(len(gt_row))]
    data_df = pd.DataFrame(data, columns=columns)
    
    # Generate a unique CSV filename
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    csv_path = os.path.join(base_dir, f"data_temp_{temp}_epoch_50_{timestamp}.csv")
    
    # Save the DataFrame to a CSV file
    data_df.to_csv(csv_path, index=False)
    print(f"Saved data to CSV at {csv_path}")

def autoregressive_generation(model, tokenizer, initial_input_str, num_steps, temp):
    current_input = extract_task(initial_input_str, end_task_token='>')
    generated_states = []

    for _ in range(num_steps):
        current_input_tensor = torch.Tensor(tokenizer.texts_to_sequences(current_input, do_padding=False)).to(device)
        current_input_tensor = current_input_tensor.transpose(0, 1).long()

        with torch.no_grad():
            sample = model.generate(
                prompts=current_input_tensor,
                seq_len=generate_length,
                temperature=temp,
                cache_kv=True
            )

        output_str = extract_sample(tokenizer.sequences_to_texts(sample[:1])[0])
        print(f"Output: {output_str}")
        generated_states.append(output_str)

        # Update the input for the next step
        current_input = f"@PredictNextState<{output_str}>"

        del sample, current_input_tensor
        torch.cuda.empty_cache()

    return generated_states


# Assuming X_data_test[2] is already defined
X_data_test_new = X_data_test[2]

# Calculate max_length and generate_length dynamically
max_length = len(X_data_test_new[0])
generate_length = max_length - len("@PredictNextState<>") - (32 * 32)

# Initialize tokenizer
tokenizer_X = Tokenizer(max_length, device)

# Load the model for epoch 50
epoch = 50
model_path = f"C:\\Users\\jaime\\ML_Playground_1\\model_parameters\\07_22_2024_Conway_2_State_Jump_Rot_Pos_On_Masking_On_Broad_Entrpoy_Homog_2024-07-23 10-37-31\\Conway_GPT_v7_epoch_{epoch}.pt"
model = load_model(model_path, max_length, num_words=256)

# Define the temperatures
temperatures = [0, 0.25, 0.5, 0.75, 1]

# Generate and save the states
num_steps = 9  # Number of steps to generate autoregressively
ground_truth_df = df_test  # Use the df_test DataFrame for ground truth data

for temp in temperatures:
    print(f"Generating for temperature {temp}...")
    
    autoregressive_states = []
    row_indices = []
    
    for i, item in enumerate(X_data_test_new):
        # Take the initial input from the test data
        initial_input_str = str(item)
        
        # Generate subsequent states
        generated_states = autoregressive_generation(model, tokenizer_X, initial_input_str, num_steps, temp)
        autoregressive_states.append(generated_states)
        row_indices.append(i)  # Assuming the index in X_data_test corresponds to the DataFrame row
        
        del generated_states
        torch.cuda.empty_cache()
    
    # Save the data for the current temperature
    save_data_to_csv(ground_truth_df, row_indices, autoregressive_states, temp)

    del autoregressive_states
    torch.cuda.empty_cache()


## Gif Generation for Epoch 50

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import seaborn as sns
from PIL import Image, ImageDraw, ImageFont

def create_gifs_from_csv(csv_path_pattern, temp, output_dir="Figures/Epoch 50 GIFs", fps=5):
    """
    Creates looping GIFs from the CSV file, showing the evolution of Ground Truth (GT), Predicted (Pred),
    and Discrepancy states together.

    Parameters:
    csv_path_pattern (str): Pattern to match the CSV files.
    temp (float): The temperature associated with the data.
    output_dir (str): Directory to save the generated GIFs.
    fps (int): Frames per second for the GIF.
    """
    # Use glob to find the file that matches the pattern
    csv_files = glob.glob(csv_path_pattern)
    
    if not csv_files:
        print(f"No CSV files found for temperature {temp}")
        return
    
    # Assuming only one file matches the pattern
    csv_path = csv_files[0]
    
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Determine the number of states and samples
    num_columns = len([col for col in df.columns if "State" in col])
    num_samples = df.shape[0] // 2  # Since we have GT and Pred rows, divide by 2
    
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_samples):
        frames = []
        for j in range(num_columns):
            fig, axes = plt.subplots(3, 1, figsize=(3, 9), dpi=300)

            gt_row = df.iloc[2 * i, 1:]  # Skip the first column which is 'Type'
            pred_row = df.iloc[2 * i + 1, 1:]

            # GT image
            plot_image(axes[0], gt_row.iloc[j], f"GT", "gray")
            # Predicted image
            plot_image(axes[1], pred_row.iloc[j], f"Predicted", "gray")
            # Discrepancy image
            discrepancy_matrix = calculate_discrepancy_matrix(gt_row.iloc[j], pred_row.iloc[j])
            plot_discrepancy(axes[2], discrepancy_matrix, f"Discrepancy")

            # Add titles
            fig.suptitle(f"Sample {i + 1} | Temperature: {temp} | Epoch: 50/50", fontsize=10)

            plt.tight_layout(pad=2.0)
            
            # Save the figure to a temporary file
            temp_file = os.path.join(output_dir, f"temp_frame_{j}.png")
            plt.savefig(temp_file, dpi=300)
            plt.close(fig)
            
            # Add the image to the frame list
            frames.append(Image.open(temp_file))

        # Create the GIF
        gif_path = os.path.join(output_dir, f"evolution_temp_{temp}_sample_{i + 1}.gif")
        frames[0].save(gif_path, save_all=True, append_images=frames[1:], duration=1000 // fps, loop=0)
        print(f"GIF saved at {gif_path}")

        # Clean up temporary files
        for frame in frames:
            os.remove(frame.filename)

def plot_image(ax, state, ylabel, cmap):
    """
    Plots an image from the state data.

    Parameters:
    ax (Axes): The axes to plot on.
    state (str): The state as a string of 0s and 1s.
    ylabel (str): The label for the y-axis.
    cmap (str): The color map for the image.
    """
    ax.set_ylabel(ylabel, fontsize=10, rotation=90, labelpad=10, va='center')
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Turn off ticks and labels
    for spine in ax.spines.values():
        spine.set_linewidth(0.5)  # Set the width of the spines
    if not is_perfect_square(len(state)) or not check_state_length(state):
        ax.text(0.5, 0.5, "DATA NOT SQUARE", fontsize=10, ha='center', va='center', rotation=45)
    else:
        ax.imshow(1 - convert_to_square_image(state, cell_size=10), cmap=cmap, vmin=0, vmax=1)

def plot_discrepancy(ax, discrepancy_matrix, ylabel):
    """
    Plots the discrepancy matrix.

    Parameters:
    ax (Axes): The axes to plot on.
    discrepancy_matrix (ndarray): The discrepancy matrix to plot.
    ylabel (str): The label for the y-axis.
    """
    sns.heatmap(discrepancy_matrix, cmap="plasma", cbar=False, square=True, ax=ax)
    ax.set_ylabel(ylabel, fontsize=10, rotation=90, labelpad=10, va='center')
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Turn off ticks and labels

def calculate_discrepancy_matrix(gt_state, pred_state):
    """
    Calculate the discrepancy matrix between GT and Predicted states.

    Parameters:
    gt_state (str): The ground truth state.
    pred_state (str): The predicted state.

    Returns:
    np.ndarray: The discrepancy matrix.
    """
    if len(gt_state) == len(pred_state):
        discrepancy_matrix = np.array([
            1 if gt_state[k] != pred_state[k] else 0
            for k in range(len(gt_state))
        ]).reshape((32, 32))
    else:
        print('not square!')
        discrepancy_matrix = np.full((32, 32), 0.5)  # Handle error gracefully
        print(discrepancy_matrix)
    
    return discrepancy_matrix

def convert_to_square_image(state, cell_size=1):
    """
    Converts a string state representation into a square image with adjustable cell size.
    
    Parameters:
    state (str): The state as a string of 0s and 1s.
    cell_size (int): The size of each cell in pixels. Default is 1.
    
    Returns:
    2D numpy array representing the square image with adjusted cell size.
    """
    length = len(state)
    size = int(np.sqrt(length))
    assert size * size == length, "State length is not a perfect square"
    image = np.array([int(pixel) for pixel in state]).reshape((size, size))
    
    # Adjust the cell size by repeating the rows and columns
    image = np.kron(image, np.ones((cell_size, cell_size)))
    
    return image

def is_perfect_square(length):
    """
    Check if a number is a perfect square.
    """
    size = int(np.sqrt(length))
    return size * size == length

def check_state_length(state):
    """
    Check if the state has 1024 entries.
    """
    return len(state) == 1024

# Example usage
temperatures = [0, 0.25, 0.5, 0.75, 1]
base_dir = "Autoregressive_autoregressor"

for temp in temperatures:
    csv_path_pattern = os.path.join(base_dir, f"data_temp_{temp}_epoch_50_*.csv")
    create_gifs_from_csv(csv_path_pattern, temp)


In [28]:
##This code looks to see if any of the testing set samples are identical to training set samples
import pandas as pd

# File paths
training_file ='Conway_GPT_v8\\conway_states_0_1_10000by32by32by10_toroidal_20240711_133408.csv'
test_file ='Conway_GPT_v8\\conway_test_states_32by32_20240716_151502.csv'

# Load the datasets, selecting only the first column
training_data = pd.read_csv(training_file, usecols=[0], header=None)
test_data = pd.read_csv(test_file, usecols=[0], header=None)

# Check for matches
matches = test_data[test_data[0].isin(training_data[0])]

if not matches.empty:
    print(f"Test samples that match training samples:\n{matches}")
else:
    print("No matching samples found.")


Test samples that match training samples:
                                                   0
0                                            State 1
1  0000000000000000000000000000000000000000000000...
5  1111111111111111111111111111111111111111111111...


Samples 1 and 5 matching is expected, as these are the 'all 0s' and 'all 1s' test-cases, which are the 1st and 10,000th samples in the training sert respectively.

In [3]:
### This is some extra code that converts gifs to mp4 filesand places them in the same folder as the gifs
import os
from moviepy.editor import VideoFileClip

def convert_gif_to_mp4(gif_path, mp4_path):
    clip = VideoFileClip(gif_path)
    clip.write_videofile(
        mp4_path, 
        codec="libx264", 
        ffmpeg_params=['-pix_fmt', 'yuv420p']
    )
    clip.close()
()

# Walk through the directory
for root, dirs, files in os.walk(search_directory):
    for file in files:
        if file.endswith('.gif'):
            gif_path = os.path.join(root, file)
            mp4_path = os.path.join(root, file.replace('.gif', '.mp4'))
            
            # Check if the corresponding MP4 already exists
            if not os.path.exists(mp4_path):
                print(f"Converting {gif_path} to {mp4_path}")
                convert_gif_to_mp4(gif_path, mp4_path)
            else:
                print(f"MP4 already exists for {gif_path}, skipping...")

print("Conversion process complete.")

Converting c:\Users\jaime\ML_Playground_1\TransformerLife\toroidal_grid_glider_gun.gif to c:\Users\jaime\ML_Playground_1\TransformerLife\toroidal_grid_glider_gun.mp4


                                                                
                                                                
t:   0%|          | 1/235 [14:04<54:53:46, 844.56s/it, now=None]

Moviepy - Building video c:\Users\jaime\ML_Playground_1\TransformerLife\toroidal_grid_glider_gun.mp4.
Moviepy - Writing video c:\Users\jaime\ML_Playground_1\TransformerLife\toroidal_grid_glider_gun.mp4














                                                                
                                                                
t:   0%|          | 1/235 [14:05<54:58:57, 845.89s/it, now=None]

Moviepy - Done !
Moviepy - video ready c:\Users\jaime\ML_Playground_1\TransformerLife\toroidal_grid_glider_gun.mp4
MP4 already exists for c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\Ground_Truth\blinkers.gif, skipping...
MP4 already exists for c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\Ground_Truth\cloverleaf.gif, skipping...
MP4 already exists for c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\Ground_Truth\glider.gif, skipping...
MP4 already exists for c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\Ground_Truth\hammer_head_spaceship.gif, skipping...
MP4 already exists for c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\Ground_Truth\order_param_0.25.gif, skipping...
MP4 already exists for c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\Ground_Truth\order_param_0.5.gif, skipping...
MP4 already exists for c:\Users\jaime\ML_Playground_1\Transformer

...

Moviepy - Done !
Moviepy - video ready c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\GT_Pred_Disc\Epoch 50\Temperature 1\evolution_temp_1_sample_8.mp4
Converting c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\GT_Pred_Disc\Epoch 50\Temperature 1\evolution_temp_1_sample_9.gif to c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\GT_Pred_Disc\Epoch 50\Temperature 1\evolution_temp_1_sample_9.mp4


                                                                
                                                                
t:   0%|          | 1/235 [15:08<59:02:50, 908.42s/it, now=None]

Moviepy - Building video c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\GT_Pred_Disc\Epoch 50\Temperature 1\evolution_temp_1_sample_9.mp4.
Moviepy - Writing video c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\GT_Pred_Disc\Epoch 50\Temperature 1\evolution_temp_1_sample_9.mp4







                                                                
                                                                
t:   0%|          | 1/235 [15:08<59:04:48, 908.93s/it, now=None]

Moviepy - Done !
Moviepy - video ready c:\Users\jaime\ML_Playground_1\TransformerLife\Testing Set Animations\GT_Pred_Disc\Epoch 50\Temperature 1\evolution_temp_1_sample_9.mp4
Conversion process complete.
